# Introduction to NeuralHydrology

**Before we start**

- This tutorial is rendered from a Jupyter notebook that is hosted on GitHub. If you want to run the code yourself, you can find the notebook and configuration files [here](https://github.com/neuralhydrology/neuralhydrology/tree/master/examples/01-Introduction).
- To be able to run this notebook locally, you need to download the publicly available CAMELS US rainfall-runoff dataset. See the [Data Prerequisites Tutorial](data-prerequisites.nblink) for a detailed description on where to download the data and how to structure your local dataset folder. You will also need to follow the [installation instructions](https://neuralhydrology.readthedocs.io/en/latest/usage/quickstart.html#installation) (the easiest option if you don't plan to implement your own models/datasets is `pip install neuralhydrology`; for other options refer to the installation instructions).

The Python package NeuralHydrology was was developed with a strong focus on research. The main application area is hydrology, however, in principle the code can be used with any data. To allow fast iteration of research ideas, we tried to develop the package as modular as possible so that new models, new data sets, new loss functions, new regularizations, new metrics etc. can be integrated with minor effort.

There are two different ways to use this package:

1. From the terminal, making use of some high-level entry points (such as `nh-run` and `nh-run-scheduler`)
2. From any other Python file or Jupyter Notebook, using NeuralHydrology's API

In this tutorial, we will give a very short overview of the two different modes.

Both approaches require a **configuration file**. These are `.yml` files which define the entire run configuration (such as data set, basins, data periods, model specifications, etc.). A full list of config arguments is listed in the [documentation](https://neuralhydrology.readthedocs.io/en/latest/usage/config.html) and we highly recommend to check this page and read the documentation carefully. There is a lot that you can do with this Python package and we can't cover everything in tutorials.

For every run that you start, a new folder will be created. This folder is used to store the model and optimizer checkpoints, train data means/stds (needed for scaling during inference), tensorboard log file (can be used to monitor and compare training runs visually), validation results (optionally) and training progress figures (optionally, e.g., model predictions and observations for _n_ random basins). During inference, the evaluation results will also be stored in this directory (e.g., test period results).


### TensorBoard logging
By default, the training progress is logged in TensorBoard files (add `log_tensorboard: False` to the config to disable TensorBoard logging). If you installed a Python environment from one of our environment files, you have TensorBoard already installed. If not, you can install TensorBoard with:

```
pip install tensorboard
``` 

To start the TensorBoard dashboard, run:

```
tensorboard --logdir /path/to/run-dir
```

You can also visualize multiple runs at once if you point the `--logdir` to the parent directory (useful for model intercomparison)

### File logging
In addition to TensorBoard, you will always find a file called `output.log` in the run directory. This file is a dump of the console output you see during training and evaluation.


## Using NeuralHydrology from the Terminal

### nh-run


Given a run configuration file, you can use the bash command `nh-run` to train/evaluate a model. To train a model, use


```bash
nh-run train --config-file path/to/config.yml
```

to evaluate the model after training, use

```bash
nh-run evaluate --run-dir path/to/run-directory
```

### nh-run-scheduler

If you want to train/evaluate multiple models on different GPUs, you can use the `nh-run-scheduler`. This tool automatically distributes runs across GPUs and starts a new one, whenever one run finishes.

Calling `nh-run-scheduler` in `train` mode will train one model for each `.yml` file in a directory (or its sub-directories).

```bash
nh-run-scheduler train --directory /path/to/config-dir --runs-per-gpu 2 --gpu_ids 0 1 2 3 
```
Use `-runs-per-gpu` to define the number of models that are simultaneously trained on a _single_ GPU (2 in this case) and `--gpu-ids` to define which GPUs will be used (numbers are ids according to nvidia-smi). In this example, 8 models will train simultaneously on 4 different GPUs.

Calling `nh-run-scheduler` in `evaluate` mode will evaluate all models in all run directories in a given root directory.

```bash
nh-run-scheduler evaluate --directory /path/to/parent-run-dir/ --runs-per-gpu 2 --gpu_ids 0 1 2 3 
```

## API usage

Besides the command line tools, you can also use the NeuralHydrology package just like any other Python package by importing its modules, classes, or functions.

This can be helpful for exploratory studies with trained models, but also if you want to use some of the functions or classes within a different codebase. 

Look at the [API Documentation](https://neuralhydrology.readthedocs.io/en/latest/api/neuralhydrology.html) for a full list of functions/classes you could use.

The following example shows how to train and evaluate a model via the API.

In [1]:
import pickle
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import torch
from neuralhydrology.evaluation import metrics
from neuralhydrology.nh_run import start_run, eval_run

### Train a model for a single config file

**Note**

- The config file assumes that the CAMELS US dataset is stored under `data/CAMELS_US` (relative to the main directory of this repository) or a symbolic link exists at this location. Make sure that this folder contains the required subdirectories `basin_mean_forcing`, `usgs_streamflow` and `camels_attributes_v2.0`. If your data is stored at a different location and you can't or don't want to create a symbolic link, you will need to change the `data_dir` argument in the `1_basin.yml` config file that is located in the same directory as this notebook.
- By default, the config (`1_basin.yml`) assumes that you have a CUDA-capable NVIDIA GPU (see config argument `device`). In case you don't have any or you have one but want to train on the CPU, you can either change the config argument to `device: cpu` or pass `gpu=-1` to the `start_run()` function.

In [2]:
# by default we assume that you have at least one CUDA-capable NVIDIA GPU
if torch.cuda.is_available():
    start_run(config_file=Path("config_mts.yml"))

# fall back to CPU-only mode
else:
    start_run(config_file=Path("config_mts.yml"), gpu=-1)

2023-08-11 23:45:27,207: Logging to /home/jonat/neuralhydrology/2023_train_lstm_for_nextgen/runs/NOAH_MP_Attributes_hourly_1108_234527/output.log initialized.
2023-08-11 23:45:27,208: ### Folder structure created at /home/jonat/neuralhydrology/2023_train_lstm_for_nextgen/runs/NOAH_MP_Attributes_hourly_1108_234527
2023-08-11 23:45:27,208: ### Run configurations for NOAH_MP_Attributes_hourly
2023-08-11 23:45:27,209: experiment_name: NOAH_MP_Attributes_hourly
2023-08-11 23:45:27,210: train_basin_file: basins.txt
2023-08-11 23:45:27,210: validation_basin_file: basins.txt
2023-08-11 23:45:27,211: test_basin_file: basins.txt
2023-08-11 23:45:27,211: train_start_date: 2000-10-01 00:00:00
2023-08-11 23:45:27,212: train_end_date: 2008-09-30 00:00:00
2023-08-11 23:45:27,213: validation_start_date: 1983-10-01 00:00:00
2023-08-11 23:45:27,213: validation_end_date: 1985-09-30 00:00:00
2023-08-11 23:45:27,214: test_start_date: 1990-10-01 00:00:00
2023-08-11 23:45:27,215: test_end_date: 1999-09-30 00

RuntimeError: Loss was NaN for 151 times in a row. Stopped training.

### Evaluate run on test set
The run directory that needs to be specified for evaluation is printed in the output log above. Since the folder name is created dynamically (including the date and time of the start of the run) you will need to change the `run_dir` argument according to your local directory name. By default, it will use the same device as during the training process.

In [ ]:
def list_subdirs(directory):
    """Return a list of subdirectory names sorted in descending order."""
    subdirs = [d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]
    return sorted(subdirs, reverse=True)


In [ ]:
run_dirs = list_subdirs("./runs/")
run_dir = Path(f"runs/{run_dirs[0]}")
#run_dir = Path(f"./runs/NOAH_MP_Attributes_hourly_1108_102550")
print(f"Evaluating run {str(run_dir)}")
eval_run(run_dir=run_dir, period="test")

### Load and inspect model predictions
Next, we load the results file and compare the model predictions with observations. The results file is always a pickled dictionary with one key per basin (even for a single basin). The next-lower dictionary level is the temporal resolution of the predictions. In this case, we trained a model only on daily data ('1D'). Within the temporal resolution, the next-lower dictionary level are `xr`(an xarray Dataset that contains observations and predictions), as well as one key for each metric that was specified in the config file.

In [ ]:
with open(run_dir / "test" / "model_epoch020" / "test_results.p", "rb") as fp:
    results = pickle.load(fp)
    
results.keys()

The data variables in the xarray Dataset are named according to the name of the target variables, with suffix `_obs` for the observations and suffix `_sim` for the simulations.

Let's plot the model predictions vs. the observations

In [ ]:
with open('basins.txt', 'r') as f:
    gauge_ids = [str(line.strip()) for line in f] 

In [ ]:
print_plot_prob = 0.02
for gauge_id in list(gauge_ids):
    if np.random.random() < print_plot_prob:
        print(gauge_id)
        # extract observations and simulations
        qobs = results[gauge_id]['1D']['xr']['qobs_mm_per_hour_obs']
        qsim = results[gauge_id]['1D']['xr']['qobs_mm_per_hour_sim']

        fig, ax = plt.subplots(figsize=(8,5))
        ax.plot(qobs['date'], qobs, label="Observation")
        ax.plot(qsim['date'], qsim, label="LSTM")
        ax.set_ylabel("Discharge (mm/d)")
        ax.set_title(f"{gauge_id}")
        plt.legend()
        plt.show()
        plt.close()


In [ ]:
metrics_dict  = {'NSE': [],
 'MSE': [],
 'RMSE': [],
 'KGE': [],
 'Alpha-NSE': [],
 'Beta-KGE': [],
 'Beta-NSE': [],
 'Pearson-r': [],
 'FHV': [],
 'FMS': [],
 'FLV': [],
 'Peak-Timing': [],
 'Peak-MAPE': []}
for gauge_id in list(gauge_ids):
    qobs = results[gauge_id]['1D']['xr']['qobs_mm_per_hour_obs']
    qsim = results[gauge_id]['1D']['xr']['qobs_mm_per_hour_sim']
    try:
        values = metrics.calculate_all_metrics(qobs.isel(time_step=-1), qsim.isel(time_step=-1))
    except:
        continue
    for key, val in values.items():
        metrics_dict[key].append(val)

In [ ]:
for key in list(metrics_dict.keys()):
    print(f"MEAN {key}: {np.mean(metrics_dict[key]):.3}")
    print(f"MEDIAN {key}: {np.median(metrics_dict[key]):.3}")
    print(f"MAX {key}: {np.max(metrics_dict[key]):.3}")
    print(f"MIN {key}: {np.min(metrics_dict[key]):.3}")

In [ ]:
print("Finished")